# Feature Engineering from Audio Files

In [1]:
import librosa
import numpy as np 
import matplotlib.pyplot as plt
import soundfile as sf
import glob
import numpy as np
import pandas as pd 
# from keras.models import Sequential
# from keras.layers import Dense
# from keras.utils.np_utils import to_categorical

### Importing the data and generating summary

In [91]:
labels = pd.read_csv('ag_doc.csv')
labels['performance_id'] = labels['performance_id'].astype(str)
labels['age'] = 2018.0-labels['birth_year']
labels = labels.sort_values(by = 'performance_id')

print(labels.isnull().sum())
labels.tail()

performance_id            0
account_id                0
country                   4
locale                    1
latitude               1585
longitude              1585
city_id               11216
gender                 7790
birth_year             8300
creation_timestamp        0
device_os                 0
headphones             3714
age                    8300
dtype: int64


,performance_id,account_id,country,locale,latitude,longitude,city_id,gender,birth_year,creation_timestamp,device_os,headphones,age
1652,99939429_92448907,99940322,US,en_US,37.78,-122.39,NaN,F,1998.0,1421451397,Android,1.0,20.0
5584,99954255_14520813,99954272,US,en_US,33.52,-101.86,NaN,F,1958.0,1375658682,iOS,NaN,60.0
11427,99962538_13150729,99960658,US,en_US,NaN,NaN,NaN,NaN,NaN,1374386655,iOS,NaN,NaN
13605,99966149_13173143,50439556,US,en_US,NaN,NaN,NaN,NaN,NaN,1374423344,iOS,NaN,NaN
1549,99990810_13161611,99998472,GB,en_GB,12.99,77.66,NaN,NaN,NaN,1374406810,iOS,NaN,NaN


### Separating labels for age and gender

In [92]:
labels_gender = labels[labels.gender.notnull()]
gender_ids = np.asarray(labels_gender.performance_id.astype(str))
gender_labs = np.asarray(labels_gender.gender.astype(str))

labels_age = labels[labels.birth_year.notnull()]
age_ids = np.asarray(labels_age.performance_id.astype(str))
age_labs = np.asarray(labels_gender.age.astype(str))

In [95]:
all_ids = np.asarray(labels['performance_id'])
all_ids

array(['100013538_14906339', '100030930_13364402', '100051581_16289560',
       ..., '99962538_13150729', '99966149_13173143', '99990810_13161611'],
      dtype=object)

### Extracting Features from Audio files

In [167]:
ctr = 0
fid = []
prob_fid = []
features = np.empty((0,195))
prob_fid1 = []

for filename in all_ids:
    try:
        clip,sr = sf.read('amazing_grace/'+filename+'.m4a')
    except: 
        print('An error occurred for: ',filename,'.wav')
        prob_fid1.append(filename)
        continue
    
    if sr not in range(12000,40000):
        print('Problematic file',filename)
        prob_fid.append(filename)
        continue
    
    mat_stft = np.abs(librosa.core.stft(y = clip, n_fft = 1024,hop_length= 512))
    
    mat_mfcc = np.mean(librosa.feature.mfcc(clip,sr,n_mfcc = 40),axis = 1)

    mat_chroma = np.mean(librosa.feature.chroma_stft(S=mat_stft, sr=sr),axis=1)

    mat_mel = np.mean(librosa.feature.melspectrogram(clip, sr=sr),axis=1)

    mat_contrast = np.mean(librosa.feature.spectral_contrast(S=mat_stft, sr=sr),axis=1)

    mat_tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(clip),
    sr=sr),axis=1)
    
    zero_cross = np.mean(librosa.feature.zero_crossing_rate(clip))
    cent = np.mean(librosa.feature.spectral_centroid(clip,sr))

    
    ext_features = np.hstack([mat_mfcc,mat_chroma,mat_mel, mat_contrast,mat_tonnetz,zero_cross,cent])
    
    with open('features_all2.csv','ab') as f:
        np.savetxt(f, ext_features, delimiter=",", fmt='%.4f')

    features = np.vstack([features, ext_features])
    
    fid.append(filename)

    ctr += 1
    
    print('Clip',ctr, ':',filename,'processed')

Clip 2961 : 162946156_38354722 processed
Problematic file 162969968_35050941
Clip 2962 : 163035225_137363870 processed
Clip 2963 : 163035225_34210047 processed
Clip 2964 : 163056250_32609289 processed
Clip 2965 : 163062397_79288864 processed
Problematic file 163096931_33852476
Problematic file 163156556_33219845
Clip 2966 : 163204697_108398657 processed
Clip 2967 : 163204697_33688696 processed
Clip 2968 : 163204697_83660512 processed
Clip 2969 : 163219275_122206448 processed
Clip 2970 : 163249304_59743344 processed
Clip 2971 : 163251916_88362824 processed
Clip 2972 : 163283246_159244759 processed
Clip 2973 : 163309970_31003516 processed
Clip 2974 : 163309970_31136875 processed
Clip 2975 : 163312579_97882921 processed
Clip 2976 : 163330264_31172019 processed
Clip 2977 : 163344248_66108717 processed
Clip 2978 : 163349752_45309583 processed
Clip 2979 : 163476184_49721788 processed
Clip 2980 : 163526335_39699387 processed
Clip 2981 : 163541822_31151228 processed
Clip 2982 : 163606263_55559

Clip 3137 : 167943416_35019103 processed
Clip 3138 : 167958230_129833528 processed
Problematic file 167959708_45093337
Clip 3139 : 167964604_280969661 processed
Clip 3140 : 167995027_33083964 processed
Clip 3141 : 168028409_112669674 processed
Clip 3142 : 168028409_32328719 processed
Clip 3143 : 168028409_42459250 processed
Clip 3144 : 168069838_32366989 processed
Clip 3145 : 168114576_90994688 processed
Clip 3146 : 168155982_104155686 processed
Clip 3147 : 168162621_42466014 processed
Clip 3148 : 168235999_32409659 processed
Clip 3149 : 168258457_32740500 processed
Clip 3150 : 168265284_89238303 processed
Clip 3151 : 168287174_34143461 processed
Clip 3152 : 168318307_62510127 processed
Clip 3153 : 168322932_87317410 processed
Problematic file 168425717_63624584
Clip 3154 : 168432287_32696612 processed
Problematic file 168434507_86671912
Clip 3155 : 168441362_32996091 processed
Clip 3156 : 168441362_33155376 processed
Clip 3157 : 168441362_35024482 processed
Clip 3158 : 168470078_35024

Clip 3309 : 172132059_44281015 processed
Clip 3310 : 172176483_33742187 processed
Clip 3311 : 172176483_34524977 processed
Clip 3312 : 172176483_34592607 processed
Clip 3313 : 172176483_39808481 processed
Clip 3314 : 172176483_42625457 processed
Clip 3315 : 172176483_47773996 processed
Clip 3316 : 172236632_67524771 processed
Clip 3317 : 172239119_122907753 processed
Clip 3318 : 172274429_36738966 processed
Clip 3319 : 172277818_52123322 processed
Clip 3320 : 172297306_119963210 processed
Clip 3321 : 172348143_140541781 processed
Clip 3322 : 172380089_42184075 processed
Clip 3323 : 172390558_38735252 processed
Clip 3324 : 172411677_212035980 processed
Clip 3325 : 172413298_34236573 processed
Clip 3326 : 172413298_60738593 processed
Clip 3327 : 172429773_39909357 processed
Clip 3328 : 172436806_34092834 processed
Clip 3329 : 172454744_52856863 processed
Clip 3330 : 172454744_61192105 processed
Clip 3331 : 172462794_61608741 processed
Clip 3332 : 172492757_99589750 processed
Clip 3333 : 

Clip 3479 : 177038125_54030703 processed
Clip 3480 : 177038125_54309811 processed
Clip 3481 : 177051924_91915609 processed
Problematic file 177060660_52944382
Problematic file 177081424_308775042
Problematic file 177111392_45256287
Clip 3482 : 177117141_35200090 processed
Clip 3483 : 177117141_36029548 processed
Clip 3484 : 177222961_122221617 processed
Clip 3485 : 177263503_35508955 processed
Problematic file 177371118_113006147
Clip 3486 : 177413722_35221825 processed
Clip 3487 : 177417486_86601215 processed
Clip 3488 : 177475658_35727634 processed
Clip 3489 : 177484881_50969749 processed
Clip 3490 : 177484881_57590828 processed
Clip 3491 : 177540364_35895918 processed
Clip 3492 : 177616937_35373998 processed
Clip 3493 : 177655318_35688817 processed
Clip 3494 : 177671071_56842139 processed
Clip 3495 : 177677044_50778381 processed
Problematic file 177682312_38494916
Clip 3496 : 177693210_35916321 processed
Clip 3497 : 177710395_35558678 processed
Clip 3498 : 177710395_35569272 process

Clip 3653 : 182769557_48025305 processed
Clip 3654 : 182769557_48473651 processed
Clip 3655 : 182769557_52970171 processed
Clip 3656 : 182772165_84332478 processed
Clip 3657 : 182808066_38972703 processed
Clip 3658 : 182852555_38796189 processed
Clip 3659 : 182852998_37200444 processed
Clip 3660 : 182894327_45981245 processed
Clip 3661 : 182930001_38342849 processed
Clip 3662 : 182931567_65342264 processed
Clip 3663 : 182962524_91510688 processed
Clip 3664 : 182981346_322089122 processed
Clip 3665 : 182983213_63316073 processed
Clip 3666 : 182994450_37065146 processed
Clip 3667 : 182994450_38188759 processed
Clip 3668 : 183022366_90444175 processed
Clip 3669 : 183054322_204986622 processed
Clip 3670 : 183059779_37536738 processed
Clip 3671 : 183074375_40172292 processed
Clip 3672 : 183086254_37490836 processed
Clip 3673 : 183131488_86015216 processed
Clip 3674 : 183137946_38462311 processed
Clip 3675 : 183160327_37755762 processed
Clip 3676 : 183170110_194610747 processed
Clip 3677 : 1

Problematic file 186910774_38732288
Clip 3835 : 186917847_38544033 processed
Clip 3836 : 186922854_102798766 processed
Clip 3837 : 187115327_117658678 processed
Clip 3838 : 187337125_40111626 processed
Clip 3839 : 187348516_39602292 processed
Clip 3840 : 187348516_39687744 processed
Clip 3841 : 187348516_39714066 processed
Clip 3842 : 187348516_39844669 processed
Clip 3843 : 187348516_40446365 processed
Clip 3844 : 187348516_40586235 processed
Clip 3845 : 187348516_41074359 processed
Clip 3846 : 187348516_43307104 processed
Clip 3847 : 187389716_89969102 processed
Clip 3848 : 187389716_90864574 processed
Clip 3849 : 187403432_61561436 processed
Problematic file 187418779_40797669
Clip 3850 : 187445463_155041382 processed
Clip 3851 : 187446725_52924925 processed
Problematic file 187469898_38637025
Clip 3852 : 187473084_38635376 processed
Clip 3853 : 187495166_206142415 processed
Clip 3854 : 187500646_126246519 processed
Problematic file 187510134_53332751
Clip 3855 : 187547661_38749151 

Clip 3999 : 192138488_91710436 processed
Clip 4000 : 192275096_268769325 processed
Clip 4001 : 192275096_283190300 processed
Clip 4002 : 192309555_46300548 processed
Problematic file 192329809_42249019
Clip 4003 : 192443827_43481532 processed
Clip 4004 : 192487111_40240641 processed
Clip 4005 : 192487111_40492220 processed
Clip 4006 : 192492095_54613233 processed
Problematic file 192508222_84199510
Problematic file 192538686_40609388
Clip 4007 : 192541495_49681609 processed
Clip 4008 : 192550531_66054513 processed
Clip 4009 : 192550531_69861324 processed
Clip 4010 : 192550531_86781133 processed
Clip 4011 : 192583238_42420687 processed
Clip 4012 : 192594348_41430258 processed
Clip 4013 : 192620623_67307152 processed
Clip 4014 : 192624011_87828816 processed
Problematic file 192624591_141820118
Problematic file 192624591_155207687
Clip 4015 : 192625004_88692632 processed
Clip 4016 : 192656114_41778441 processed
Clip 4017 : 192670539_40300160 processed
Clip 4018 : 192696982_40308574 proces

Clip 4165 : 198421203_72515729 processed
Clip 4166 : 198506832_66426260 processed
Clip 4167 : 198518638_100021424 processed
Clip 4168 : 198608382_42214590 processed
Clip 4169 : 198625712_77251813 processed
Clip 4170 : 198649019_55813696 processed
Clip 4171 : 198649573_164047422 processed
Clip 4172 : 198650140_42911979 processed
Problematic file 198655744_68350398
Clip 4173 : 198659246_89835205 processed
Problematic file 198663752_48863751
Clip 4174 : 198716467_57816130 processed
Clip 4175 : 198741014_43006548 processed
Clip 4176 : 198790217_47204421 processed
Problematic file 198797593_43360744
Clip 4177 : 198831339_43490267 processed
Problematic file 198841280_47311662
Clip 4178 : 198844436_42678592 processed
Problematic file 198858600_47146897
Clip 4179 : 198859552_43474863 processed
Clip 4180 : 199048749_181435512 processed
Clip 4181 : 199060821_139482098 processed
Clip 4182 : 199061866_56708449 processed
Clip 4183 : 199085550_44335937 processed
Clip 4184 : 199098119_138971187 proce

### Converting features to a DataFrame

In [173]:
features_df = pd.DataFrame(features)
features_df['performance_id'] = fid


In [176]:
features_df

,0,1,2,3,4,5,6,7,8,9,...,186,187,188,189,190,191,192,193,194,performance_id
0,-258.196775,93.455719,-10.094956,2.026917,-12.351987,-14.215934,-9.354964,-15.423727,-30.071764,-5.131339,...,26.631288,-0.004101,-0.001844,0.014276,0.016673,0.000789,-0.001875,0.095424,2182.753054,100013538_14906339
1,-460.667475,24.563885,12.583078,-1.752247,4.303905,-27.978977,-24.924845,-20.891908,-16.000752,-13.382666,...,31.782816,0.000336,0.006147,0.015078,0.010199,0.000005,-0.003233,0.124145,3482.688630,100030930_13364402
2,-251.666166,99.236268,-46.683074,-8.256827,-31.420451,-33.318703,-24.912724,5.338037,-26.475699,-14.077890,...,31.744913,-0.004196,0.016818,0.005504,0.001501,-0.002293,0.000257,0.090770,1886.944314,100051581_16289560
3,-253.348278,56.606612,-3.405210,4.532725,-14.656238,-3.339739,-29.638392,-9.891059,-19.007195,-18.010067,...,35.103417,-0.008035,0.008099,0.034291,0.007322,-0.003011,0.000716,0.114655,2571.532160,100112831_13200625
4,-293.166536,81.802191,-10.629579,18.070702,-33.462648,-23.472141,-0.972550,-31.375172,-9.770726,-12.386659,...,29.914533,-0.008880,0.002776,0.049073,0.015104,0.002078,0.003383,0.103055,2243.467529,100118262_13452888
5,-302.947073,56.318431,-17.786253,-11.122579,-22.194047,-23.081034,-17.476873,-14.193215,-19.445069,-10.518538,...,49.703886,-0.004940,0.010082,0.010223,0.017418,-0.002217,0.000332,0.171064,2751.255383,100119127_13195184
6,-284.650565,48.832288,-17.253327,8.850252,-29.334627,-41.352264,-19.729207,-10.902560,-10.881862,-15.008690,...,27.830176,-0.006580,0.001005,-0.002714,0.000173,-0.001463,-0.003747,0.117339,2146.452815,100125241_14688700
7,-351.850280,85.140010,-4.649776,3.937031,1.389846,-8.383493,-20.072938,-11.694147,-15.294674,-16.254382,...,48.432928,-0.002645,-0.003958,0.011292,-0.056445,0.004219,0.010784,0.114964,2091.107659,100128262_13604411
8,-328.511302,65.542910,-8.370999,-1.818467,-0.359365,-24.942012,-10.541211,-15.980264,-15.083216,-3.591638,...,45.388582,-0.002670,-0.003843,-0.031129,-0.033160,0.001954,-0.013564,0.115548,2682.797659,100144633_13209180
9,-334.335372,67.087579,-5.579044,0.173670,-3.898167,-23.191553,-12.940804,-14.344381,-12.645951,-2.956795,...,23.218699,-0.004757,-0.004355,-0.008594,-0.042531,-0.005205,-0.022432,0.112389,2564.730313,100144633_13209683


### Saving the Feature files

In [177]:
features_df.to_csv('features_all_0_5000.csv')

In [190]:
np.savetxt('problematic_ids1.csv', prob_fid, delimiter=",", fmt="%s")

### Appending labels to the features

In [191]:
new_df = pd.read_csv('features_all_0_10000.csv')
new_df.head()

,performance_id,0,1,2,3,4,5,6,7,8,...,185,186,187,188,189,190,191,192,193,194
0,100013538_14906339,-258.196775,93.455719,-10.094956,2.026917,-12.351987,-14.215934,-9.354964,-15.423727,-30.071764,...,24.785505,26.631288,-0.004101,-0.001844,0.014276,0.016673,0.000789,-0.001875,0.095424,2182.753054
1,100030930_13364402,-460.667475,24.563885,12.583078,-1.752247,4.303905,-27.978977,-24.924845,-20.891908,-16.000752,...,21.593978,31.782816,0.000336,0.006147,0.015078,0.010199,0.000005,-0.003233,0.124145,3482.688630
2,100051581_16289560,-251.666166,99.236268,-46.683074,-8.256827,-31.420451,-33.318703,-24.912724,5.338037,-26.475699,...,31.937576,31.744913,-0.004196,0.016818,0.005504,0.001501,-0.002293,0.000257,0.090770,1886.944314
3,100112831_13200625,-253.348278,56.606612,-3.405210,4.532725,-14.656238,-3.339739,-29.638392,-9.891059,-19.007195,...,25.042222,35.103417,-0.008035,0.008099,0.034291,0.007322,-0.003011,0.000716,0.114655,2571.532160
4,100118262_13452888,-293.166536,81.802191,-10.629579,18.070702,-33.462648,-23.472141,-0.972550,-31.375172,-9.770726,...,27.668685,29.914533,-0.008880,0.002776,0.049073,0.015104,0.002078,0.003383,0.103055,2243.467529


In [193]:
gender_features_df = pd.concat([new_df,labels],axis = 1, join = 'inner')
gender_features_df.shape

(8446, 209)

In [195]:
gender_features_df.head()

,performance_id,0,1,2,3,4,5,6,7,8,...,locale,latitude,longitude,city_id,gender,birth_year,creation_timestamp,device_os,headphones,age
0,100013538_14906339,-258.196775,93.455719,-10.094956,2.026917,-12.351987,-14.215934,-9.354964,-15.423727,-30.071764,...,en_US,33.69,-116.43,NaN,F,NaN,1368076074,iOS,NaN,NaN
1,100030930_13364402,-460.667475,24.563885,12.583078,-1.752247,4.303905,-27.978977,-24.924845,-20.891908,-16.000752,...,en_US,34.13,-97.36,NaN,NaN,NaN,1369029990,iOS,NaN,NaN
2,100051581_16289560,-251.666166,99.236268,-46.683074,-8.256827,-31.420451,-33.318703,-24.912724,5.338037,-26.475699,...,en_GB,51.37,-0.05,NaN,F,1999.0,1371374336,iOS,NaN,19.0
3,100112831_13200625,-253.348278,56.606612,-3.405210,4.532725,-14.656238,-3.339739,-29.638392,-9.891059,-19.007195,...,en_US,35.14,-84.90,4658861.0,F,1992.0,1377362111,iOS,NaN,26.0
4,100118262_13452888,-293.166536,81.802191,-10.629579,18.070702,-33.462648,-23.472141,-0.972550,-31.375172,-9.770726,...,en_US,NaN,NaN,NaN,F,1991.0,1379121587,iOS,NaN,27.0


### Saving the final DataFrame for age and gender

In [199]:
gender_features_only = gender_features_df[gender_features_df['gender'].notnull()]
gender_features_only.to_csv('gender_features_with_labs_10000.csv')

In [200]:
age_features_only = gender_features_df[gender_features_df['age'].notnull()]
age_features_only.to_csv('age_features_with_labs_10000.csv')